In [877]:
%%capture
import numpy as np
import pandas as pd
import re
import random
!pip install pymorphy2
import pymorphy2
import dill
import os

import matplotlib.pyplot as plt
import matplotlib.lines as lines
import matplotlib.text as text
import matplotlib.cm as cm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from google.colab import drive
drive.mount('/content/drive')
# Исправьте путь до папки на вашем гугл диске:
gc_path = r"/content/drive/MyDrive/Хакатон_2022/"

In [878]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Входные тренировочные данные. 

In [879]:
df = pd.read_excel(gc_path+'data_train_1000.xlsx')
num_classes = 15
class_names = ['Сроки','Документы','Места','Проходной и допустимый балл','Достижения','Льготы','Общежития','Оплата','Другие вопросы','Аспирантура','Перевод','Ошибки','Забрать заявление','Магистратура','Очно-заочное и зачное']
df

,CONTENT,TYPE,CLASS
0,"Здравствуйте! Есть ли специальности, где добав...",Горячая линия,4
1,Здравствуйте! Задавала вопрос о консультации к...,Горячая линия,8
2,Здравствуйте! Я хотел бы задать вопрос о поряд...,Виртуальная приемная,3
3,Здравствуйте! В 2021 году есть ли набор на спе...,Горячая линия,2
4,"Здравствуйте, какое самое большое количество б...",Горячая линия,3
...,...,...,...
962,Здравствуйте! В том году меня отчислили. Мне н...,Виртуальная приемная,8
963,Здравствуйте!Возможен ли перевод на целевое ил...,Виртуальная приемная,10
964,Я родился в России и у меня есть российское гр...,Горячая линия,8
965,Добрый день! Сегодня на сайте ПетрГУ была разм...,Обращения в Совет ветеранов,8


In [880]:
stop_words_greetings = ['спасибо','подскажите','расскажите','скажите','объясните','пожалуйста', 'здравствуйте','здравствуйте!','добрый день','до свидания', 'доброго времени суток', 'доброго дня', 'hello', 'С уважением']
cls_dic = {1:['Верный класс'],
           0:['Неверный класс']}

## Преобразование меток классов, например, из 2 5 в 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 

In [881]:
def OHE_labels(df):
  temp = []
  for cl in df['CLASS']:
    new_type = np.array([0]*num_classes)
    classes = cl.split(' ')
    for i in classes:
      new_type[int(i)] = 1
    temp.append(new_type)
  y = np.array(temp)
  y = y.transpose()

  for nom, class_name in enumerate(['Сроки','Документы','Места','Проходной и допустимый балл','Достижения','Льготы','Общежития','Оплата','Другие вопросы','Аспирантура','Перевод','Ошибки','Забрать заявление','Магистратура','Очно-заочное и зачное']):
    df[class_name] = y[nom]
  return df

In [882]:
df = OHE_labels(df)

# Подзадача 1. Выбор/Разработка методов предварительной обработки текста
## Подготовка данных

In [883]:
def re_gr(txt):
    '''
    Удаление приветсвий и стоп-слов из строки.
    '''
    for x in stop_words_greetings:
        txt = txt.replace(x,'')
    return txt

def del_punct(txt):
    '''
    Удаляет знаки препинания и подчеркивания.
    '''
    res = re.sub(r'[^\w\s]',' ',txt)
    res = res.replace('_','')
    res = res.replace('\n','')
    return res

def morphan(word, morph):
    '''
    Приведение слов в нормальную форму, удаление числительных и ФИО.
    '''
    word = del_punct(word).strip()
    p = morph.parse(word)[0]
   
    word_new = word    
    if (not 'Surn' in p.tag) and (not 'Name' in p.tag) and (not 'Patr' in p.tag) and ('NOUN' in p.tag): 
        #существительное не ФИО
        word_new = p.normal_form
     
    elif 'Surn' in p.tag:
        word_new = 'ФАМИЛИЯ'
    elif 'Name' in p.tag:
        word_new = 'ИМЯ'
    elif 'Patr' in p.tag:
        word_new = 'ОТЧЕСТВО'
       
    elif ('INFN' in p.tag) or ('VERB' in p.tag): #глагол
        word_new = p.normal_form
                
    elif ('ADJF' in p.tag) or ('ADJS' in p.tag) or ('COMP' in p.tag): #прилагательное
        word_new = p.normal_form
            
    elif ('PRTF' in p.tag) or ('PRTS' in p.tag) or ('GRND' in p.tag): #причастие, похоже на глагол
        word_new = p.normal_form
                
    elif ('ADVB' in p.tag) or ('NPRO' in p.tag) or ('PRED' in p.tag) or ('PREP' in p.tag) or ('CONJ' in p.tag) or ('PRCL' in p.tag) or ('INTJ' in p.tag):
        # предлоги, местоимения и пр.
        word_new = p.normal_form         
  
    elif ('NUMR' in p.tag) or ('NUMB' in p.tag) or ('intg' in p.tag): # числительные NUMB,intg
        word_new = ''
        
    else:
        word_new = word

    return word_new

def remove2(txt):
    '''
    :txt - текст
    Возвращает слова, в которых не менее 2 символов.
    '''
    #ПОВЫШЕНИЕ ЭТО ЗЛО!
    return ' '.join([x if len(x)>2 else '' for x in txt.split()])

def normtext(txt, morph):
    '''
    Возвращает текст из слов в нормальной форме
    '''
    return str(' '.join([morphan(x, morph) for x in txt.split()]))


In [884]:
def prepare_dataframe(df):
  df['text'] = df['CONTENT'].map(lambda x: x.lower()).map(re_gr)
  df['text'] = df.text.map(del_punct)
  morph = pymorphy2.MorphAnalyzer()
  df['text'] = df.text.map(lambda x: normtext(x, morph))
  df['text'] = df.text.map(remove2)
  return df


In [885]:
df = prepare_dataframe(df)

# Подзадача 2. Выбор/Разработка способа векторизации текста
## Tfidf Vectorizer (получение векторного представления)


In [886]:
tfv = TfidfVectorizer()  # Функция получения векторного представления
tfv.fit(df.text)

TfidfVectorizer()

In [887]:
# Функция для нахождения оптимальных значений параметров классификатора:

def get_best_classifier(X_train, y_train, classifier, param_grid):
  X_train = tfv.transform(X_train)  
  clf = GridSearchCV(classifier, param_grid, scoring='f1', n_jobs=-1) #, cv=3)
  clf.fit(X_train, y_train)
  return clf.best_estimator_

In [888]:
def predictor(text, clf, tfv):
    '''
    Предсказывает класс сообщения
    :param text: классифицируемый текс
    :param clf: обученный классификатор
    :param tfv: обученный векторизатор
    '''
    X_test = tfv.transform([text])
    
    pred = clf.predict(X_test)
    
    return int(pred[0])

In [889]:
# Сериализация / десериализция модели
def serialize(obj, filename):
    '''
    Сериализация объекта в файл
    '''
    output = open(filename, 'wb')
    dill.dump(obj, output)
    output.close()
    return
 
def deserialize(filename):
    '''
    Десериализация объекта из файла
    '''
    input = open(filename, 'rb')
    obj = dill.load(input)
    input.close()
    return obj

# Классификация

In [890]:
def get_binary_datasets(df, num):
  '''
  :param df: Входной DataFrame
  :param num: Искомый (верный) класс
  :return Функция возвращает новый Binary DataFrame, в котором 1 означает, что сообщение относиться к соотвествующему классу, 0 - иначе
  '''
  temp_Content = []
  temp_Class = []

  for i, cl in enumerate(df['CLASS']):
    classes_str = cl.split(' ')
    classes_int = [int(x) for x in classes_str]
    temp_Content.append(df.iloc[i][18])  # 18 - номер столбца с обработанным сообщением
    if num in classes_int:
      temp_Class.append(1)
    else:
      temp_Class.append(0)

  content = np.array(temp_Content)
  classes = np.array(temp_Class)

  data = pd.concat([pd.DataFrame(content), pd.DataFrame(classes)], axis=1)
  data.columns = ['CONTENT','CLASS']
  data.to_excel('output.xlsx')

  return data

## Делим входной набор данных на тренировочные и валидацонные данные

In [891]:
X_train, X_valid, y_train_del, y_valid_del = train_test_split(df, df['CLASS'], random_state=42, test_size=0.2) 

In [892]:
train_binary_dataframes = []
valid_binary_dataframes = []
for i in range(num_classes):
  train_binary_dataframes.append(get_binary_datasets(X_train,i))
  valid_binary_dataframes.append(get_binary_datasets(X_valid,i))

# Разработка классификатора сообщений на основе одного метода ML/DL, настройка параметров классификатора
## Обучение моделей

In [893]:
for nom in range(num_classes):
  # Задаем классифкатор
  # classifier = SVC(max_iter=10000)  # !!! Эксперименты
  # Задаем словарь парамтеров с их значениями для перебора
  # Находим оптимальный классифкатор (класификтор с оптимальным значением параметров)
  classifier = LogisticRegression(class_weight="balanced")#, cache_size=1024)#kernel='sigmoid'), probability=True)  # !!! Эксперименты
  # Задаем словарь парамтеров с их значениями для перебора
  param_grid = {}#'C': [3., 100.],
  #               # 'gamma': ['scale', 'auto', [3., 100.]]
  #               # 'degree':[2, 100]
  #               'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], #, 'precomputed'] не считает с ним потому что должно быть просчитано заранее?
  #               # 'verbose': [False, True],
  #               'probability': [False, True],
  #               # 'decision_function_shape' works only with ovr
  #               # 'cache_size': [100, 4096]
  #               # 'tol': [1e-100, 1e-99]
                 

                
  # !!! Эксперименты

  clf = get_best_classifier(train_binary_dataframes[nom].CONTENT, train_binary_dataframes[nom].CLASS, classifier, param_grid)
  clf_tfv = {'clf':clf,'tfv':tfv}

  serialize(clf_tfv, gc_path + f'models/model_{nom}_class.pkl')

  # Предсказываем
  pred = []
  for txt in valid_binary_dataframes[nom].CONTENT: #.values:
    pred.append(predictor(txt, clf, tfv))

  # Выводим метрики классификации каждого бинарного классификатора
  mtrs = metrics.classification_report([cls_dic[x][0] for x in valid_binary_dataframes[nom].CLASS], [cls_dic[x][0] for x in pred])
  print(f"Metrics for {nom} class")
  print(mtrs, end='\n')
  

Metrics for 0 class
                precision    recall  f1-score   support

  Верный класс       0.54      0.59      0.57        22
Неверный класс       0.95      0.94      0.94       172

      accuracy                           0.90       194
     macro avg       0.74      0.76      0.75       194
  weighted avg       0.90      0.90      0.90       194

Metrics for 1 class
                precision    recall  f1-score   support

  Верный класс       0.42      0.40      0.41        25
Неверный класс       0.91      0.92      0.91       169

      accuracy                           0.85       194
     macro avg       0.66      0.66      0.66       194
  weighted avg       0.85      0.85      0.85       194

Metrics for 2 class
                precision    recall  f1-score   support

  Верный класс       0.83      0.56      0.67         9
Неверный класс       0.98      0.99      0.99       185

      accuracy                           0.97       194
     macro avg       0.91      0.78 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for 5 class
                precision    recall  f1-score   support

  Верный класс       0.00      0.00      0.00         3
Неверный класс       0.98      1.00      0.99       191

      accuracy                           0.98       194
     macro avg       0.49      0.50      0.50       194
  weighted avg       0.97      0.98      0.98       194

Metrics for 6 class
                precision    recall  f1-score   support

  Верный класс       1.00      1.00      1.00         5
Неверный класс       1.00      1.00      1.00       189

      accuracy                           1.00       194
     macro avg       1.00      1.00      1.00       194
  weighted avg       1.00      1.00      1.00       194

Metrics for 7 class
                precision    recall  f1-score   support

  Верный класс       0.60      0.43      0.50         7
Неверный класс       0.98      0.99      0.98       187

      accuracy                           0.97       194
     macro avg       0.79      0.71 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  UserWarning,


Metrics for 9 class
                precision    recall  f1-score   support

  Верный класс       1.00      0.50      0.67         2
Неверный класс       0.99      1.00      1.00       192

      accuracy                           0.99       194
     macro avg       1.00      0.75      0.83       194
  weighted avg       0.99      0.99      0.99       194

Metrics for 10 class
                precision    recall  f1-score   support

  Верный класс       0.75      0.82      0.78        11
Неверный класс       0.99      0.98      0.99       183

      accuracy                           0.97       194
     macro avg       0.87      0.90      0.88       194
  weighted avg       0.98      0.97      0.97       194

Metrics for 11 class
                precision    recall  f1-score   support

  Верный класс       0.50      0.53      0.51        19
Неверный класс       0.95      0.94      0.95       175

      accuracy                           0.90       194
     macro avg       0.72      0.7

# Многометочная классификация

In [894]:
# Т.к. вопрорсов по аспирантуре мало, и в них в основном встречается подстрока 'аспирант', то можно записи проклассифицировать по наличию этой подстроки
def find_aspirant(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'аспирант'
    '''
    return 'аспирант' in [x[:8] for x in txt.split()]

In [895]:
def find_oplata(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'оплат'
    '''
    return 'оплата' in [x[:6] for x in txt.split()]

In [896]:
def find_magistr(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'магистр'
    '''
    return 'магистр' in [x[:7] for x in txt.split()]

In [897]:
def find_och(txt):
    '''
    Возвращает True если в тексте есть слово, начинающееся на 'очн'
    '''
    return 'очн' in [x[:3] for x in txt.split()]

## Возможно так и с другими классами поступить можно (как с классом "аспирантура")?

# Валидация модели

In [898]:
# Функция, которая предсказывает все метки сообщений
def pred_labels(messages=None):
  preds = []
  txts = {}
  for txt in messages:
    pred = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    skip = []
    if find_aspirant(txt):
      skip.append(9)
      pred[9] = 1
    # if find_oplata(txt):
    #   skip.append(7)
    #   pred[7] = 1
    # if find_magistr(txt):
    #   skip.append(13)
    #   pred[13] = 1
    # if find_och(txt):
    #   skip.append(14)
    #   pred[14] = 1
    for i in range(num_classes):
      if i in skip:
        continue
      clf_tfv = deserialize(gc_path + f'models/model_{i}_class.pkl')
      if predictor(txt, clf_tfv['clf'], clf_tfv['tfv']) == 1:
        pred[i] = 1
      else:
        pred[i] = 0
    preds.append(pred)
  return preds

In [899]:
# Предсказываем все метки сообщений
preds = pred_labels(messages=X_valid.text)

In [900]:
true_labels = X_valid[class_names].values

## Матрицы ошибок. Раскомментировать при необходимости:

In [901]:
# from sklearn.metrics import multilabel_confusion_matrix
# vis_arr = multilabel_confusion_matrix(true_labels, preds)

# labels = ["".join("" + str(i)) for i in range(1, 16)]

# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
# vis_arr[7]

In [902]:
# from sklearn.metrics import multilabel_confusion_matrix
# vis_arr = multilabel_confusion_matrix(true_labels, preds)

# labels = ["".join("" + str(i)) for i in range(1, 16)]

# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns


# def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=18):

#     df_cm = pd.DataFrame(
#         confusion_matrix, index=class_names, columns=class_names,
#     )

#     try:
#         heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
#         sns.set(font_scale = 1.6)
#     except ValueError:
#         raise ValueError("Confusion matrix values must be integers.")
#     heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
#     heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
#     axes.set_ylabel('True class', fontsize=fontsize)
#     axes.set_xlabel('Predicted class', fontsize=fontsize)
#     axes.set_title("class_" + class_label, fontsize=fontsize)
  

# fig, ax = plt.subplots(15,1,figsize=(4, 15))
# for axes, cfs_matrix, label in zip(ax.flatten(), vis_arr[0:15], labels[0:15]):
#     print_confusion_matrix(cfs_matrix, axes, label, ["False", "True"])
# fig.subplots_adjust(left=None, bottom=1, right=None, top=3, wspace=None, hspace=2)
# fig.tight_layout()
# plt.show()

In [903]:
def get_f1_score(true_labels, preds):
  f1 = []
  y_true_all = np.array(true_labels)
  y_pred_all = np.array(preds)
  for i in range(num_classes):
    y_true = y_true_all[:, i]
    y_pred = y_pred_all[:, i]
    f1.append(metrics.f1_score(y_true, y_pred, average='weighted'))
  print('FinAL f1 = ', sum(f1)/len(f1))

In [904]:
get_f1_score(true_labels, preds)

FinAL f1 =  0.9420969724707927


# Тестирование модели
! Файл для тестирования data_test.xlsx будет выложен на гугл диск задачи 7 августа в 8:30.

In [905]:
df_test = pd.read_excel(gc_path+'data_test.xlsx')
df_test = prepare_dataframe(df_test)
df_test = OHE_labels(df_test)

In [906]:
test_preds = pred_labels(messages=df_test.text)

In [907]:
test_true_labels = df_test[class_names].values

In [908]:
get_f1_score(test_true_labels, test_preds)

FinAL f1 =  0.9325686671969158
